# Monitoring Environmental Waste Utilization Scores
# A guide for generating EWU-Dashboards
**Sarah Schmidt & David Laner**  
*January 2023*

## 1. Configuration

### 1.1 Import packages

In [3]:
import pandas as pd
import numpy as np
import brightway2 as bw
import premise
from mycolorpy import colorlist as mcp
import string

import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

In [4]:
# create a list with MS Excel column indices (A, B, ..., Z, AA, AB, ...)
# will be used for preparing calculations in MS Excel
alphabet = list(string.ascii_uppercase)
excel_cols=[]

for i in range(100):
    n=0
    j=i
    while j-len(alphabet)>=0:
        j=j-len(alphabet)
        n=n+1
    if n>0:
        col=alphabet[n-1]+alphabet[j]
    else:
        col=alphabet[j]
    excel_cols.append(col)

In [8]:
conversion_factors={'kg':1,
                    't':1e3,
                    'kt':1e6,
                    'g':1e-3,
                    'Mg':1e3,
                    'Gg':1e6}

### 1.2 Gather case study-specific information

In [10]:
# read various case study specific information from input data file (ExcelTool_GeneratorInput_Template.xlsx)
general_info=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='GeneralInformation', index_col=0)
general_info=general_info[general_info.columns[0]].to_dict()

Pathway_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_codes=Pathway_codes[Pathway_codes['Pathway tag'].notnull()]['Activity code'].to_list()
Pathway_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
Pathway_names=Pathway_names[Pathway_names['Pathway tag'].notnull()]['Pathway tag'].to_list()

WasteGen_code=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
WasteGen_code=WasteGen_code[WasteGen_code['Waste generation'].notnull()]['Activity code'].iloc[0]

act_codes=[WasteGen_code,*Pathway_codes]
act_names=[general_info['Acronym'],*Pathway_names]

# materials for calculation of the environmental impact of materials
material_names=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_names=material_names['Material Name'].to_list()
material_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='EnvironmentalValue')
material_codes=material_codes['Material Code'].to_list()

waste_utilization_codes=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
waste_utilization_codes=waste_utilization_codes[pd.isnull(waste_utilization_codes['Waste utilization'])==False]['Activity code'].to_list()

# activity tags for contribution analysis
activity_tags=pd.read_excel('ExcelTool_GeneratorInput_Template.xlsx', sheet_name='Activities')
activity_tags=activity_tags['Activity tag'].to_list()
activity_tags=[tag for tag in activity_tags if pd.isnull(tag)==False]
activity_tags=list(set(activity_tags))
activity_tags.append('Others')

# waste quantity in kg
WQ_kg=general_info['Waste quantity']*conversion_factors[general_info['Unit']]

### 1.3 Create databases